# Roulette-Verzierung

In [2]:
%matplotlib inline
from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

con = sqlite3.connect("../data/CongoDB.sqlite")

wie viel macht Roulette-Verzierung an alle aufgenommenen Verzierungselementen aus:

In [2]:
# Alle Verzierungen in der DB aus eigenen Fpl.
sql = """SELECT
           t_Obj.objID,
           [t_K_Pos].[posID] || ' ' || [posName] AS pos,
           t_K_Verz.verzName AS verzName,
           t_K_Verz.verzBeschr
       FROM ((t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID) t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE ((t_Ort.ort_Lit) = 'DS')""".replace('\n',' ')

df = pd.read_sql(sql, con)

In [3]:
a = pd.Series({'Roulette':len(df.loc[df['verzName'].str.contains("V08")].index),
               'Sonstige Verzierungen':len(df.loc[~df['verzName'].str.contains("V08")].index)})
a = pd.DataFrame(a)
a['Prozent'] = a[0]/a[0].sum()*100
a

,0,Prozent
Roulette,765,9.309967
Sonstige Verzierungen,7452,90.690033


## Roulette-Verzierungen

In [7]:
# Alle Roulette-Verzierungen in der DB aus eigenen Fpl.
sql = """SELECT
           t_Obj.objID AS objID,
           [t_K_Pos].[posID] || ' ' || [posName] AS pos,
           t_K_Verz.verzName AS verzName,
           t_K_Verz.verzBeschr
       FROM ((t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID) t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE (((t_Ort.ort_Lit) = 'DS')
           AND ((t_K_Verz.verzName) Like '%21.%'))""".replace('\n',' ')

df = pd.read_sql(sql, con)

In [8]:
df_pivot = df.pivot_table(values = 'objID', index = 'verzName', aggfunc = len)
df_pivot = pd.DataFrame(df_pivot)

In [10]:
fig = plt.figure()
fig.set_size_inches(5, 5)
df_pivot['objID'].plot(kind='pie', autopct='%.0f %%', cmap = 'Paired')
plt.ylabel('')

## Kommen Vegetabilisches- und Schnitzroulette einmal gemeinsam auf einem Gefäß vor?

In [2]:
# Alle Verzierungen in der DB aus eigenen Fpl.
sql = """SELECT
           t_Obj.objID as Obj,
           [t_Ort].[ort_kurz] || ' ' || [t_Komplex].[bef_nr] || ':' || [t_obj].[Individuum] AS INDIV,
           [t_K_Pos].[posID] || ' ' || [posName] AS pos,
           t_K_Verz.verzName AS verzName,
           t_K_Verz.verzBeschr
       FROM ((t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID) t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE (((t_Ort.ort_Lit) = 'DS')
           AND ((t_K_Verz.verzName) = '21.1')
           OR ((t_K_Verz.verzName) = '21.2')
           OR ((t_K_Verz.verzName) = '21.3')
           OR ((t_K_Verz.verzName) = '21.4'))""".replace('\n',' ')

df1 = pd.read_sql(sql, con)
df1['tech'] = 'vegetabilisch'

sql = """SELECT
           t_Obj.objID as Obj,
           [t_Ort].[ort_kurz] || ' ' || [t_Komplex].[bef_nr] || ':' || [t_obj].[Individuum] AS INDIV,
           [t_K_Pos].[posID] || ' ' || [posName] AS pos,
           t_K_Verz.verzName AS verzName,
           t_K_Verz.verzBeschr
       FROM ((t_Ort LEFT JOIN t_Komplex ON t_Ort.ortID = t_Komplex.ortID)
           LEFT JOIN t_Obj ON t_Komplex.komplexID = t_Obj.komplexID) t_Obj INNER JOIN "t_ObjPosVerz" ON t_Obj.objID = "t_ObjPosVerz".objID
           INNER JOIN t_K_Pos ON "t_ObjPosVerz".posID = t_K_Pos.posID
           INNER JOIN t_K_Verz ON "t_ObjPosVerz".verzID = t_K_Verz.verzID
       WHERE (((t_Ort.ort_Lit) = 'DS')
           AND ((t_K_Verz.verzName) Like '21.%')
           AND ((t_K_Verz.verzName) != '21.1')
           AND ((t_K_Verz.verzName) != '21.2')
           AND ((t_K_Verz.verzName) != '21.3')
           AND ((t_K_Verz.verzName) != '21.4'))""".replace('\n',' ')

df2 = pd.read_sql(sql, con)
df2['tech'] = 'schnitz'

df = pd.concat([df1, df2])

display(df.head())

,Obj,INDIV,pos,verzName,t_K_Verz.verzBeschr,tech
0,3631,NGL 87/101:,1 Pos unbekannt,21.1,vegetabilisches Roulette/Knotted String,vegetabilisch
1,3630,NGL 87/101:,1 Pos unbekannt,21.1,vegetabilisches Roulette/Knotted String,vegetabilisch
2,1356,NGL 87/101:58,1 Pos unbekannt,21.1,vegetabilisches Roulette/Knotted String,vegetabilisch
3,1353,NGL 87/101:67,1 Pos unbekannt,21.1,vegetabilisches Roulette/Knotted String,vegetabilisch
4,1351,NGL 87/101:74,1 Pos unbekannt,21.1,vegetabilisches Roulette/Knotted String,vegetabilisch


In [3]:
df_pivot = df.pivot_table(values = 'pos', index = ['Obj', 'INDIV'], columns = 'tech', aggfunc = len, margins = 'TRUE')
df_pivot = pd.DataFrame(df_pivot)
# nur jene Stücke, bei denen beide typen vertreten sind
# ansonsten ist eine der spalten lee
df_pivot.dropna()

,tech,schnitz,vegetabilisch,All
Obj,INDIV,,,
5362,MTB 85/101:11,2,1,3
5567,MAO 85/101:65,2,1,3
5574,MAO 85/101:46,1,1,2
5751,LIB 85/101:7,1,1,2
5836,BAT 85/101:56,1,1,2
5866,"MBK 85/101:49, 64, 76",1,2,3
5875,MBK 85/101:80,2,2,4
All,,441,350,791


Nach Anne Mayor (04.12.2015) gehen vegetabilisches und Schnitz-Roulette überhaupt nicht zusammen. Das hier muss also herausgestellt werden! Oder habe ich die evtl nur falsch bestimmt?

> [ ] Nachsehen, ob die Stücke gezeichnet sind und wenn ja, dann heraussuchen.